In [1]:
from __future__ import print_function

import psycopg2
import datetime
import sys
from operator import itemgetter, attrgetter, methodcaller
import numpy as np
import itertools
import os.path
import matplotlib.pyplot as plt
import math
from multiprocessing import Pool, cpu_count
import re
import traceback
import random
import shutil

from utils import getConnection
from utils import parseUnitsMap
from utils import parseNum
from utils import sparsify

random.seed(0)

%matplotlib inline

# How to use code in this folder

This folder generates materialized views for a variety of severity of illness scores (detailed below).
The queries make use of materialized views which aggregate data from the first day of a patient's ICU stay. To run the code, it is necessary to:

1. Run the scripts which generate the constituent materialized views
    * echodata - generated by /concepts/echo-data.sql 
    * ventdurations - generated by /concepts/ventilation-durations.sql - (needed for subsequent view)
    * vitalsfirstday - generated by /concepts/firstday/vitals-first-day.sql 
    * uofirstday - generated by /concepts/firstday/urine-output-first-day.sql
    * ventfirstday - generated by /concepts/firstday/ventilated-first-day.sql
    * gcsfirstday - generated by /concepts/firstday/gcs-first-day.sql
    * labsfirstday - generated by /concepts/firstday/labs-first-day.sql
    * bloodgasfirstday - generated by /concepts/firstday/blood-gas-first-day.sql - (needed for subsequent view)
    * bloodgasfirstdayarterial - generated by /concepts/firstday/blood-gas-first-day-arterial.sql
2. Run the script for the severity of illness score you are interested in
    * OASIS - oasis.sql
    * SAPS - saps.sql
    * SOFA - sofa.sql

The scripts were written and tested in PostgreSQL 9.4.4. The scripts are written to follow the SQL standard as close as possible, and there should be minimal changes necessary for them to operate under different languages. Key changes likely involve:
  * Changing "with" clause to subqueries if using MySQL
  * Modifying the regular expression in the *echo* view in sofa.sql as appropriate

If you modify a script to operate in another RDBMS, we would welcome a pull request!


In [2]:
conn = getConnection()
cur = conn.cursor()
working_dir = './mimic-code/'

# prepare necessary materialized views

sqlfilelist = [
    'concepts_48/echo-data.sql',
    'concepts_48/ventilation-durations.sql',
    'concepts_48/firstday/vitals-first-day.sql',
    'concepts_48/firstday/urine-output-first-day.sql',
    'concepts_48/firstday/ventilation-first-day.sql',
    'concepts_48/firstday/gcs-first-day.sql',
    'concepts_48/firstday/labs-first-day.sql',
    'concepts_48/firstday/blood-gas-first-day.sql',
    'concepts_48/firstday/blood-gas-first-day-arterial.sql'
]

for sqlfile in sqlfilelist:
    pstr = os.path.join(working_dir, sqlfile)
    if not os.path.exists(pstr):
        print(pstr)

for sqlfile in sqlfilelist:
    print('executing {0}...'.format(sqlfile))
    with open(os.path.join(working_dir, sqlfile), 'r') as f:
        sql = f.read()
        cur.execute(sql)
        conn.commit()
    print('finish executing {0}!'.format(sqlfile))
    
conn = getConnection()
cur = conn.cursor()
# sapsii
with open(os.path.join(working_dir, 'concepts_48/severityscores/sapsii.sql'), 'r') as f:
    cur.execute(f.read())
    conn.commit()

# sofa
with open(os.path.join(working_dir, 'concepts_48/severityscores/sofa.sql'), 'r') as f:
    cur.execute(f.read())
    conn.commit()

executing concepts_48/echo-data.sql...
finish executing concepts_48/echo-data.sql!
executing concepts_48/ventilation-durations.sql...
finish executing concepts_48/ventilation-durations.sql!
executing concepts_48/firstday/vitals-first-day.sql...
finish executing concepts_48/firstday/vitals-first-day.sql!
executing concepts_48/firstday/urine-output-first-day.sql...
finish executing concepts_48/firstday/urine-output-first-day.sql!
executing concepts_48/firstday/ventilation-first-day.sql...
finish executing concepts_48/firstday/ventilation-first-day.sql!
executing concepts_48/firstday/gcs-first-day.sql...
finish executing concepts_48/firstday/gcs-first-day.sql!
executing concepts_48/firstday/labs-first-day.sql...
finish executing concepts_48/firstday/labs-first-day.sql!
executing concepts_48/firstday/blood-gas-first-day.sql...
finish executing concepts_48/firstday/blood-gas-first-day.sql!
executing concepts_48/firstday/blood-gas-first-day-arterial.sql...
finish executing concepts_48/firstd

In [3]:
HRS = 48
TARGETDIR = '../../Data/admdata_17f/'
HRDIR = os.path.join(TARGETDIR, '%dhrs' % HRS)
RESDIR = os.path.join(HRDIR, 'non_series')

data_all = np.load(os.path.join(HRDIR, 'DB_merged_%dhrs.npy' % HRS)).tolist()
valid_aids = [t[0][-1] for t in data_all]
print(len(valid_aids))
print(valid_aids)

31912
[100001, 100006, 100007, 100009, 100010, 100011, 100012, 100016, 100021, 100024, 100028, 100030, 100031, 100034, 100035, 100036, 100037, 100041, 100045, 100046, 100050, 100053, 100058, 100059, 100065, 100071, 100074, 100075, 100078, 100085, 100087, 100095, 100098, 100099, 100108, 100109, 100112, 100113, 100114, 100116, 100118, 100119, 100124, 100126, 100131, 100132, 100136, 100138, 100141, 100142, 100146, 100154, 100156, 100160, 100161, 100166, 100168, 100173, 100177, 100179, 100184, 100186, 100191, 100195, 100199, 100203, 100206, 100210, 100215, 100217, 100223, 100224, 100225, 100230, 100234, 100242, 100247, 100249, 100250, 100253, 100262, 100263, 100265, 100271, 100277, 100281, 100282, 100283, 100284, 100286, 100290, 100292, 100295, 100297, 100298, 100300, 100301, 100302, 100303, 100307, 100311, 100312, 100313, 100321, 100324, 100326, 100328, 100335, 100338, 100342, 100343, 100349, 100350, 100360, 100364, 100367, 100372, 100375, 100379, 100387, 100388, 100392, 100394, 100395, 1

In [6]:
# get sapsii scores and sofa scores

conn = getConnection()
cur = conn.cursor()

# for sapsii scores, we have the sapsii and sapsii_prob
def get_sapsii(aid):
    conn = getConnection()
    sql = 'select * from mimiciii.sapsii_48 where hadm_id = {0}'.format(aid)
    cur = conn.cursor()
    cur.execute(sql)
    res = cur.fetchone()
    if res:
        return res
    else:
        return None

# for sofa scores, we only have the score and we need lr to process
def get_sofa_score(aid):
    conn = getConnection()
    sql = 'select sofa from mimiciii.sofa_48 where hadm_id = {0}'.format(aid)
    cur = conn.cursor()
    cur.execute(sql)
    res = cur.fetchone()
    if res:
        return res[0]
    else:
        return None

# print('sapsii...')
# # store sapsii subscores in file
# p = Pool(16)
# ress = [p.apply_async(get_sapsii, args=(aid,)) for aid in valid_aids]
# p.close()
# p.join()
# ress = np.array([x.get() for x in ress])
# print(ress[:10])
# np.savez(os.path.join(RESDIR, 'sapsii.npz'), sapsii=ress)

print('sofa...')
# store sofa scores in file
p = Pool(16)
ress = [p.apply_async(get_sofa_score, args=(aid,)) for aid in valid_aids]
p.close()
p.join()
ress = np.array([x.get() for x in ress])
print(ress[:10])
np.savez(os.path.join(RESDIR, 'sofa.npz'), sofa=ress)

sofa...
[2 0 2 5 2 3 6 6 4 8]


In [8]:
len(valid_aids)

31912

In [ ]:
# get sofa score: do logistic regression on sofa score
label_mor = np.load(os.path.join(HRDIR, 'ADM_LABELS_%dhrs.npy' % HRS)).tolist()
label_mor = [l[0] for l in label_mor]

from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
X = np.array(np.load(os.path.join(RESDIR, 'sofa.npz'))['sofa']).reshape(-1, 1)
y = np.array(label_mor)
lr.fit(X, y)
score = lr.score(X, y)
prob = lr.predict_proba(X)
print(score)
print(lr.classes_, prob)
sofa_score = [p[1] for p in prob]

np.savez(os.path.join(RESDIR, 'sofa_res.npz'), sofa_score=sofa_score)